In [1]:
# 함수 내부에 다른함수 정의
def greet(name):
    def get_message():
        return "Hello "

    result = get_message()+name
    return result

greet("John")

'Hello John'

In [2]:
# 다른 함수에 의한 리턴
def greet(name):
   return "Hello " + name 

def call_func(func): # 인자로 들어오는 어떤 함수던 그곳에 인자로 존을 넣는다
    other_name = "John"
    return func(other_name)

call_func(greet) 

'Hello John'

In [3]:
# p_decorate 에서 인자값으로 get_text(name)를 받은 후에 func_wrapper(sp) 를 실행 -> get_text 실행 -> 결과값 리턴 
def p_decorate(func): 
   def func_wrapper(name):
       return f"<p>{func(name)}</p>"
   return func_wrapper

@p_decorate # == p_decorate(func)
def get_text(name): # p_decorate
   return f"lorem ipsum, {name} dolor sit amet"
get_text('sp')

'<p>lorem ipsum, sp dolor sit amet</p>'

In [4]:
def p_decorate(func):
   def func_wrapper(name):
       return "<p>{0}</p>".format(func(name))
   return func_wrapper

def strong_decorate(func):
    def func_wrapper(name):
        return "<strong>{0}</strong>".format(func(name))
    return func_wrapper

def div_decorate(func):
    def func_wrapper(name):
        return "<div>{0}</div>".format(func(name))
    return func_wrapper
a = div_decorate(p_decorate(strong_decorate(get_text)))
a('hi')

'<div><p><strong><p>lorem ipsum, hi dolor sit amet</p></strong></p></div>'

In [5]:
@div_decorate # 1
@p_decorate # 2
@strong_decorate
def get_text(name):
   return "lorem ipsum, {0} dolor sit amet".format(name)

get_text("John")

'<div><p><strong>lorem ipsum, John dolor sit amet</strong></p></div>'

In [6]:
def p_decorate(func):
   def func_wrapper(self):
       return "<p>{0}</p>".format(func(self)) # get_full name의 리턴값을 여기에 넣는다
   return func_wrapper

class Person(object):
    def __init__(self):
        self.name = "John"
        self.family = "Doe"

    @p_decorate
    def get_fullname(self):
        return self.name+" "+self.family

my_person = Person()
my_person.get_fullname()

'<p>John Doe</p>'

In [7]:
def tags(tag_name):
    def tags_decorator(func):
        def func_wrapper(name):
            return "<{0}>{1}</{0}>".format(tag_name, func(name))
        return func_wrapper
    return tags_decorator

@tags("네네네")
def get_text(name):
    return "Hello "+name

get_text("John")

'<네네네>Hello John</네네네>'

In [8]:
# debugging .. due to wrapper overriding
get_text.__name__

'func_wrapper'

In [1]:
from functools import wraps

def tags(tag_name): # 데코선언, 데코 인자 받고
    def tags_decorator(func): # 위에 함수 받고
        @wraps(func) # 원하는 함수 에다가 wraps
        def func_wrapper(name): # 함수 전후로 할거 정하고
            return "<{0}>{1}</{0}>".format(tag_name, func(name))
        return func_wrapper
    return tags_decorator

@tags("ㅘㅘㅘㅘㅘㅘ")
def get_text(name):
    """returns some text"""
    return "Hello "+name

print(get_text.__name__) # get_text
print(get_text.__doc__)  # returns some text
get_text.__module__ # __main__
get_text('my name is sp')

get_text
returns some text


'<ㅘㅘㅘㅘㅘㅘ>Hello my name is sp</ㅘㅘㅘㅘㅘㅘ>'

In [10]:
def exception_handler(func):
    def inner_function(*args, **kwargs):
        try:
            func(*args, **kwargs)
        except Exception as e:
            print(f"{func.__name__} only takes numbers as the argument {e}")
    return inner_function

@exception_handler
def area_circle(radius):
    print(3.14 * radius * radius)


@exception_handler
def area_rectangle(length, breadth):
    print(length * breadth)


area_circle(2)
area_rectangle(2, 4)
area_circle("some_other_str")
area_rectangle("some_other_rectangle")

12.56
8
area_circle only takes numbers as the argument can't multiply sequence by non-int of type 'float'
area_rectangle only takes numbers as the argument area_rectangle() missing 1 required positional argument: 'breadth'


In [11]:
def enter_exit_info(func):
    def wrapper(self, *arg, **kw):
        print('-- entering', func.__name__)
        for i in list(range(5)):
          print(i)
          try:
            res = func(self, *arg, **kw)
          except Exception as e:
            print(e)
            continue
        print('-- exiting', func.__name__)
        return res
    return wrapper


class TestWrapper:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        self.c = 0

    @enter_exit_info
    def add_in_c(self):
        self.c = self.a + self.b
        print(self.c)

    @enter_exit_info
    def mult_in_c(self):
      self.c = self.a * self.b
      print(self.c)


# t = TestWrapper(2, 3)
# t.add_in_c()
# t.mult_in_c()

class b(TestWrapper):
  @enter_exit_info
  def mult_in_c(self):
      raise KeyError('ihihi')
      self.c = self.a * self.b
      print(self.c)
a = b(10, 10)
a.mult_in_c()

-- entering mult_in_c
0
'ihihi'
1
'ihihi'
2
'ihihi'
3
'ihihi'
4
'ihihi'
-- exiting mult_in_c


UnboundLocalError: local variable 'res' referenced before assignment

partial(func, args, *keywords) 호출될 때 위치 인자 args와 키워드 인자 keywords로 호출된 func처럼 동작하는 새 partial 객체를 반환합니다. 더 많은 인자가 호출에 제공되면, args에 추가됩니다. 추가 키워드 인자가 제공되면, keywords를 확장하고 대체합니다. 대략 다음과 동등합니다:

In [16]:
def partial(func, *args, **keywords):
    def newfunc(*fargs, **fkeywords):
        newkeywords = {**keywords, **fkeywords}
        return func(*args, *fargs, **newkeywords)
    newfunc.func = func
    newfunc.args = args
    newfunc.keywords = keywords
    return newfunc

# def square(base):
#     return power(base, 2)

# def cube(base):
#     return power(base, 3)

def power(base, exponent):
    return base ** exponent


square = partial(power, exponent=2)
cube = partial(power, exponent=3)


In [17]:
square(5)

25

In [18]:
cube(10)

1000

In [19]:
from functools import partial
square = partial(power, exponent=2)
square(5)

25